# Celebrity Mentions
*Example Usage of Scraipe Library*

Here's a quick example using scraipe to extract mentions of celebrities in news articles.

## Setup
Install and import things we need. We will use `NewsScraper` and `OpenAiAnalyzer` from the `scraipe[extended]` subpackage.

We will also load [your OpenAI API key](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key) from a file called 'openai_key.txt'. This file should be in the same folder as this notebook.

In [ ]:
# Install scraipe package from repo
%pip install -qe ..
# Alternatively, install scraipe from PyPI:
#%pip install --upgrade --quiet scraipe

# Check package version
!pip show scraipe | grep Version

# Install notebook widgets for cleaner output
%pip install --quiet ipywidgets

Note: you may need to restart the kernel to use updated packages.
Version: 0.0.0
Note: you may need to restart the kernel to use updated packages.


In [2]:

# Import modules
import pandas as pd
from scraipe import Workflow
from scraipe.extended import NewsScraper, OpenAiAnalyzer
from pydantic import BaseModel

# Load OpenAI API key
OPENAI_API_KEY = open("openai_key.txt").read().strip()

## Extract links
First, we need a list of links to target with scraipe. We will extract all links from the front page of https://apnews.com.

In [4]:
import requests
import re

url = "https://apnews.com/"
response = requests.get(url)
html = response.text

# Use a regex to extract article links
pattern = r'href=["\'](?=[^"\']*/article)([^"\']+)["\']'
news_links = re.findall(pattern, html)

# Remove duplicates
news_links = list(set(news_links))

# Display a summary of the links
news_links_df = pd.DataFrame(news_links, columns=['link'])
import time
print(f"Found {len(news_links_df)} front page AP News links on {time.strftime('%Y-%m-%d')}")
display(news_links_df)

Found 139 front page AP News links on 2025-05-10


,link
0,https://apnews.com/article/xi-beijing-min-aung...
1,https://apnews.com/article/pope-leo-xiv-humor-...
2,https://apnews.com/article/papa-leon-vision-pa...
3,https://apnews.com/article/rusia-ucrania-guerr...
4,https://apnews.com/article/smokey-robinson-la-...
...,...
134,https://apnews.com/article/trump-mass-firings-...
135,https://apnews.com/article/spring-cleaning-wor...
136,https://apnews.com/article/immigration-ice-new...
137,https://apnews.com/article/ispace-moon-lander-...


## Configure Workflow
Now we'll configure the scraipe workflow using `NewsScraper` and `OpenAiAnalyzer`.

`NewsScraper` uses `trafilatura` to extract article content from a news site without all the HTML clutter.

`OpenAiAnalyzer` uses OpenAI models to extract data from the article content.

In [5]:
#===Configure NewsScraper===
# NewsScraper doesn't require any additional configuration
scraper = NewsScraper()

#===Configure OpenAiAnalyzer===
# Define the instruction for the LLM. Ensure the instruction specifies a return schema.
instruction = '''
Extract a list of celebrities mentioned in the article text.
Return a JSON dictionary with the following schema:
{"celebrities":["celebrity1", "celebrity2", ...]}
'''

# (Optional) Create a pydantic schema to validate the LLM output
from typing import List
class ExpectedOutput(BaseModel):
    celebrities: List[str]
    
# Create the analyzer with the API key, instruction, and schema
analyzer = OpenAiAnalyzer(OPENAI_API_KEY,instruction,pydantic_schema=ExpectedOutput)

#===Create Workflow===
# Create a workflow with the configured scraper and analyzer
workflow = Workflow(scraper, analyzer)

## Scrape links
Next we will scrape content from news links. This content will be saved within the workflow's scrape store.

In [6]:
# Scrape the news links
workflow.scrape(news_links)
# Display the scraped content
display(workflow.get_scrapes())

Scraping:   0%|          | 0/139 [00:00<?, ?link/s]

,link,content,scrape_success,scrape_error,metadata
0,https://apnews.com/article/bangladesh-hasina-y...,Bangladesh’s interim government bans the forme...,True,None,None
1,https://apnews.com/article/syrian-bahrain-alsh...,Syria’s president visits Bahrain and discusses...,True,None,None
2,https://apnews.com/article/zealand-snail-egg-n...,A rare New Zealand snail is filmed for the fir...,True,None,None
3,https://apnews.com/article/north-carolina-elec...,Ousted top election official in key swing stat...,True,None,None
4,https://apnews.com/article/infant-mortality-cd...,US infant mortality dropped in 2024. Experts p...,True,None,None
...,...,...,...,...,...
134,https://apnews.com/article/measles-texas-vacci...,None,False,Failed to get page: Failed to scrape https://a...,None
135,https://apnews.com/article/vatican-new-pope-le...,None,False,Failed to get page: Failed to scrape https://a...,None
136,https://apnews.com/article/workplace-pandemic-...,None,False,Failed to get page: Failed to scrape https://a...,None
137,https://apnews.com/article/columbus-statue-new...,None,False,Failed to get page: Failed to scrape https://a...,None


## Analyze content
Next we will extract celebrities mentioned in each article using OpenAI

In [8]:
# Analyze the scraped content
workflow.analyze()

# Display the analyses
display(workflow.get_analyses())

Analyzing: 0link [00:00, ?link/s]

,link,output,analysis_success,analysis_error
0,https://apnews.com/article/bangladesh-hasina-y...,"{'celebrities': ['Sheikh Hasina', 'Muhammad Yu...",True,None
1,https://apnews.com/article/syrian-bahrain-alsh...,"{'celebrities': ['Donald Trump', 'Emmanuel Mac...",True,None
2,https://apnews.com/article/zealand-snail-egg-n...,{'celebrities': []},True,None
3,https://apnews.com/article/north-carolina-elec...,{'celebrities': []},True,None
4,https://apnews.com/article/infant-mortality-cd...,{'celebrities': []},True,None
...,...,...,...,...
123,https://apnews.com/article/disabled-federal-wo...,"{'celebrities': ['Donald Trump', 'Robert F. Ke...",True,None
124,https://apnews.com/article/browns-sanders-dill...,"{'celebrities': ['Shedeur Sanders', 'DeAndre C...",True,None
125,https://apnews.com/article/pulitzer-prizes-202...,"{'celebrities': ['Elon Musk', 'Greg Tate']}",True,None
126,https://apnews.com/article/brasil-amazonia-cat...,"{'celebrities': ['Francisco', 'Robert Prevost'...",True,None


## Compile the results
Finally, let's export the completed analysis. 

In [9]:
export_df = workflow.export()
display(export_df)

,link,celebrities
0,https://apnews.com/article/bangladesh-hasina-y...,"[Sheikh Hasina, Muhammad Yunus]"
1,https://apnews.com/article/syrian-bahrain-alsh...,"[Donald Trump, Emmanuel Macron]"
2,https://apnews.com/article/zealand-snail-egg-n...,[]
3,https://apnews.com/article/north-carolina-elec...,[]
4,https://apnews.com/article/infant-mortality-cd...,[]
...,...,...
134,https://apnews.com/article/measles-texas-vacci...,NaN
135,https://apnews.com/article/vatican-new-pope-le...,NaN
136,https://apnews.com/article/workplace-pandemic-...,NaN
137,https://apnews.com/article/columbus-statue-new...,NaN


## Analyze the results
Now you can conduct your own analysis on the structured data collected by the scraipe workflow.

In [ ]:
# Explode the nested list of celebrities
export_df = export_df.explode('celebrities')
export_df['celebrities'] = export_df['celebrities'].str.strip()

# Display the top 10 most mentioned celebrities
export_df = export_df['celebrities'].value_counts().reset_index()
export_df.columns = ['celebrity', 'mentions']
export_df = export_df.sort_values('mentions', ascending=False)
export_df.head(10)

,celebrity,mentions
0,Donald Trump,35
2,Robert F. Kennedy Jr.,7
1,Joe Biden,7
3,Scott Bessent,5
4,Pope Leo XIV,5
5,Robert Prevost,5
6,Marco Rubio,5
7,Shehbaz Sharif,4
9,JD Vance,4
10,Pope Francis,4
